This notebook scrapes satellite images for each leak repair. For each location it gets a NxM rectangle around the leak before and after it was repaired. Then it collated all the data into h5 files and all the metadata into json files.

It takes days to run because of rate limiting on the google earth api. Because of limited satelite coverage you might find matches for only 10% of the leaks.

## Modifying

- make sure google earth is setup
- load leaks, so they pass the asserts
- change params
- run rest of cells

In [1]:
from path import Path
import arrow
import json
import pytz
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
import re, os, collections, itertools, uuid, logging
import tempfile
import tables

import zipfile
import urllib

import ee
import pyproj
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (15, 5) # bigger plots
plt.style.use('fivethirtyeight')
%matplotlib inline
%precision 4

'%.4f'

In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
helper_dir = str(Path('..').abspath())
if helper_dir not in os.sys.path:
    os.sys.path.append(helper_dir)
    
from leak_helpers.earth_engine import display_ee, get_boundary, tifs2np, bands_s2, download_image, bands_s2, bands_s1, bands_l7, bands_l8

In [79]:

crs_grid = 3857
notebook_name='testing_earth_engine-l8-AUTX_v2'

satellite = 'LANDSAT/LC8_L1T'
bands = bands_l8

# since the lowest res band is 60m and I want to capture neighbours I should get 6+ pixels
pixel_length = 25.0
resolution_min = 15.0 # m
time_bin_delta = 60*60*24*28 # how long before a leak to look (in seconds)
# TODO get closest but let me filter for time

# init
ts=arrow.utcnow().format('YYYYMMDD-HH-mm-ss')
data_dir = Path('../../data/')
temp_dir = Path(tempfile.mkdtemp(prefix=notebook_name+'-', suffix='-'+ts))
output_dir = Path('../../data/scraped_satellite_images/20170314-05-26-52_testing_earth_engine-l8-AUTX_v2')
cache_dir = output_dir.joinpath('ee_l8_AUTX-leaks_cache_v2/ATX_')

output_dir.makedirs_p()
temp_dir.makedirs_p()
cache_dir.makedirs_p()

logger = logging.getLogger(notebook_name)
logger.setLevel(logging.WARN)

crs_grid_proj = pyproj.Proj('+init=epsg:%s'%crs_grid)

temp_dir, output_dir, cache_dir

(Path('/tmp/testing_earth_engine-l8-AUTX_v2-82dzkbkv-20170321-02-12-16'),
 Path('../data/20170314-05-26-52_testing_earth_engine-l8-AUTX_v2'),
 Path('../data/20170314-05-26-52_testing_earth_engine-l8-AUTX_v2/ee_l8_AUTX-leaks_cache_v2/ATX_'))

In [80]:
# write metadata to json
metadata = dict(
    satellite=satellite,
    notebook_name=notebook_name,
    time_bin_delta=time_bin_delta,
    pixel_length=pixel_length,
    resolution_min=resolution_min,
    bands=bands,
    ts=ts,
    crs_grid=crs_grid,
    cache_dir=str(cache_dir),
    temp_dir=str(temp_dir),
    output_dir=str(output_dir),
)
metadata_file = output_dir.joinpath('script_metadata.json')
json.dump(metadata, open(metadata_file,'w'))

# earth engine

Setup instructions here
- first need to apply for an account and wait ~ 1day
- https://developers.google.com/earth-engine/python_install#setting-up-authentication-credentials

Refs:
- api https://developers.google.com/earth-engine/
- code examples https://code.earthengine.google.com/
- sentinel1 https://developers.google.com/earth-engine/sentinel1
    - `ee.ImageCollection('COPERNICUS/S2_GRD');`
    - `ee.ImageCollection('COPERNICUS/S1_GRD');`
- keras and google earth https://github.com/patrick-dd/landsat-landstats

In [81]:
# test earth-engine setup
from oauth2client import crypt # should have not error
import ee
ee.Initialize() # should give no errors, if so follow instructions


# test
image = ee.Image('srtm90_v4')
assert image.getInfo()=={'type': 'Image', 'properties': {'system:time_start': 950227200000, 'system:asset_size': 18827626666, 'system:time_end': 951177600000}, 'bands': [{'data_type': {'type': 'PixelType', 'max': 32767, 'min': -32768, 'precision': 'int'}, 'crs': 'EPSG:4326', 'id': 'elevation', 'dimensions': [432000, 144000], 'crs_transform': [0.000833333333333, 0.0, -180.0, 0.0, -0.000833333333333, 60.0]}], 'id': 'srtm90_v4', 'version': 1463778555689000}
print('ok')

# ee.Geometry.Point([117.21079620254062, -30.94712385398404])

ok


# Load leaks

In [82]:
# load wa leaks
leaks = gpd.read_file(data_dir.joinpath('leak_datasets/austin_leaks/derived/austin_leaks-repairs.geojson'))


# they have to be after launch
s3_launch_ts=pd.Timestamp('Apr 11, 2013')
leaks = leaks[pd.to_datetime(leaks.COMPDTTM)>=s3_launch_ts]

leaks.index = leaks.leak_id
len(leaks)

7910

In [83]:
# choose one leak for now
leak = leaks.sample()
leak

,22,ADDRKEY,CITY,COMPDTTM,DESCRIPT,FullStreetName,INITDTTM,LOC,OBJECTID,PREDIR,...,REPO_Date,STNAME,STNO,STSUB,SUFFIX,WONO,ZIP,geometry,id,leak_id
leak_id,,,,,,,,,,,,,,,,,,,,,
ATX-66276,980381.0,624836.0,AUSTIN,2014-12-23T05:36:00,WATER MAIN LEAK,501 CONGRESS AVE,2014-11-19T20:09:00,IN ALLEYWAY BY BRAZOS,66276,,...,2014-12-23T05:36:00,CONGRESS,501,,AVE,1546404.0,78701-,POINT (-97.7426138688088 30.26731104576194),66276,ATX-66276


In [84]:
leaks['REPO_Date']=leaks['COMPDTTM']
leaks['leak_id']=leaks.id.apply(lambda x:'ATX_%s'%x)

# Fetching sentinal-1 and sentinel 2 images

For a leak repair, grab the image before and after it

Note roughly 10% have results for a 1 day temporal bin

In [95]:
def get_cached_ids():
    cache_dirs = []
    for f in cache_dir.listdir():
        f=f.relpath(cache_dir)
        f=Path(f.replace('ATX-','').replace('ATX_',''))
        n=str(f).split('_')[0]
        cache_dirs.append(n)
    return cache_dirs

def init_cache(leak_id):
    """We will cache downloads in folders like 'id_after'"""
    if leak_id:
        cache_subdir = cache_dir.joinpath(leak_id+'_after')
        cache_subdir.makedirs_p()
        cache_subdir = cache_dir.joinpath(leak_id+'_before')
        cache_subdir.makedirs_p()
    return get_cached_ids()

For each point
- find the nearest image before the repair
- and the soonest image after repair
- save a part of each with metadata

Later we can filter, interpolate, and read into numpy arrays

In [11]:
distance = resolution_min*(pixel_length/2.0-0.5)

In [13]:
# test with one image
for i in (np.random.sample(5)*len(leaks)).astype(np.int):
    leak=leaks.iloc[[i]]
    leak_id = str(leak.leak_id.values[0])

    repo_date_ts = arrow.get(leak.REPO_Date.values[0]).timestamp
    boundary = get_boundary(leak, distance=distance)
    sentinel2_before = ee.ImageCollection(satellite)\
        .filterBounds(boundary)\
        .filterDate(933828614605,1488776737937)\
        .sort('system:time_start', opt_ascending=False) # first will be latest
    image = ee.Image(sentinel2_before.first()).clip(boundary)
    image.getInfo()
    name=leak_id+'_after'
    path,files=download_image(
        image, 
        scale=resolution_min, 
        crs=crs_grid, 
        name=name,
        cache_dir=cache_dir
    )
    data = tifs2np(path,files,bands=bands_l7)
    print(i, [(d.shape,d.sum()) for d in data])
    for d in data:
        assert d.shape[0]==pixel_length, 'the downloaded image is the wrong size, tweak distance'
        assert d.shape[1]==pixel_length
    assert np.sum(data)!=0,'should not be empty (make sure you are using the right bands)'

2040 [((25, 25), 7535399.0), ((25, 25), 14426899.0), ((25, 25), 6713718.0), ((25, 25), 6654434.0), ((25, 25), 8400010.0), ((25, 25), 0.0), ((25, 25), 0.0), ((25, 25), 6832393.0), ((25, 25), 6751405.0)]


KeyboardInterrupt: 

In [14]:
# np.argwhere(leaks.leak_id==57914)
# leaks.leak_id.index.values

In [134]:
import time, traceback
cached_ids = get_cached_ids()

def get_image_for_leak(i, cached_ids=cached_ids):    
    leak = leaks.loc[[i]]
    repo_date_ts = arrow.get(leak.REPO_Date.values[0]).timestamp
    
    # crappy way or recording that we tried this one
    leak_id = str(leak.leak_id.values[0])
    if leak_id in cached_ids:
        logger.info('Skipping cached download for leak id %s ',leak_id)
        return
    
    boundary = get_boundary(leak, distance=distance)
    
    # get image day before    
    sentinel2_before = ee.ImageCollection(satellite)\
        .filterBounds(boundary)\
        .filterDate((repo_date_ts-time_bin_delta)*1000,(repo_date_ts)*1000)\
        .sort('system:time_start', opt_ascending=False) # first will be latest
    
    results = sentinel2_before.size().getInfo()
    if results<1:
        logger.info('Error no results for day before %s',leak_id)
        cached_ids = init_cache(leak_id) # so we know there where no results
        return
        
    # get image day after
    sentinel2_after = ee.ImageCollection(satellite)\
        .filterBounds(boundary)\
        .filterDate((repo_date_ts)*1000,(repo_date_ts+time_bin_delta*6)*1000)\
        .sort('system:time_start', opt_ascending=True) # first will be earliest
        
    results = sentinel2_after.size().getInfo()
    if results<1:
        logger.info('Error no results for day after, id %s',leak_id)
        cached_ids = init_cache(leak_id) # so we know there where no results
        return
        
    # download as save images    
    logger.info('results for %s', leak_id)
    image = ee.Image(sentinel2_before.first()).clip(boundary)
    name=leak_id+'_before'
    path,files=download_image(
        image, 
        scale=resolution_min, 
        crs=crs_grid, 
        name=name,
        cache_dir=cache_dir
    )
    # also save metadata so we can filter by date
    with open(path.joinpath('metadata.json'), 'w') as fo:
        metadata = dict(
            image=image.getInfo(),
            scale=resolution_min,
            crs=crs_grid,
            name=name,
            distance=distance,
            leak=json.loads(leak.to_json())
        )
        json.dump(metadata, fo)

    image = ee.Image(sentinel2_after.first()).clip(boundary)
    name=leak_id+'_after'
    path,files=download_image(
        image, 
        scale=resolution_min, 
        crs=crs_grid, 
        name=name,
        cache_dir=cache_dir
    )
    with open(path.joinpath('metadata.json'), 'w') as fo:
        metadata = dict(
            image=image.getInfo(),
            scale=resolution_min,
            crs=crs_grid,
            name=name,
            distance=distance,
            leak=json.loads(leak.to_json())
        )
        json.dump(metadata, fo)
        
# could take 27 hours
leak_to_scrape = set(leaks.leak_id).difference(set(['ATX_'+i for i in cached_ids]))
# leak_to_scrape = set(leaks.id).difference(set(cached_ids))
for i in tqdm(leak_to_scrape):
    try:
        get_image_for_leak(i)
    except urllib.error.HTTPError as e:
        print(i,e) # "HTTP Error 429: unknown"
        traceback.print_stack()
        if e.code == 429:
            print('sleep for 13s')
            time.sleep(13);
    except ee.ee_exception.EEException as e:
        print(i,e) # "Earth Engine memory capacity exceeded."
        traceback.print_stack()
        ee.Initialize()
    except zipfile.BadZipFile as e:
        print(i,e) # "File is not a zip file"
        traceback.print_stack()
    except Exception as e:
        print(i,e)
        # e.g. "An internal server error has occurred (216bc442fe171620592bc53fb578bceb)."
        traceback.print_stack()

# parsing tiffs

In [135]:
# This loads it as X and y for machine learning, and also time and metadata so we can filter
import shapely
X = []
y = []
t = []
m = []
discarded=[]
cdirs = [cdir for cdir in cache_dir.listdir() if ('_after_' in cdir) or ('_before_' in cdir)]
for path in tqdm(cdirs):
    if not path.isdir(): continue
    files = [file.relpath(path) for file in path.listdir() if file.endswith('.tif')]
    if files:
        # check metadata
        try:
            metadata = json.load(open(path.joinpath('metadata.json')))
        except (FileNotFoundError, ValueError) as e:
            path.move(path.dirname().dirname().joinpath('.deleteme-'+str(uuid.uuid4())))
            if '_after_' in path: # also delete the before path                
                path_after = Path(path.replace('_after_','_before_'))
                if path_after.isdir():
                    path_after.move(path.dirname().dirname().joinpath('.deleteme-'+str(uuid.uuid4())))
            logger.error('Invalid metadata.json, deleted folder %s, please rerun scraping cell to rescrape this image', path)
            continue
        
        # e.g. lets filter it so "before" image are only 1 day before
        if '_before_' in path.basename():
            yy = True
        else:
            yy = False
        
        # work out time gap too
        t1 = arrow.get(metadata['image']['properties']['system:time_end']/1000)
        t0 = arrow.get(metadata['leak']['features'][0]['properties']['REPO_Date'])
        td=t1-t0
        tt = td.total_seconds()
        
        # load data
        data = tifs2np(path,files,bands=bands)
             
        # check we don't have empty bands 1-13
        empty_bands = np.array([d.sum() for d in data])==0
        
        # lets check we didn't get the edge of an image
        bbox = np.array(metadata['image']['properties']['system:footprint']['coordinates'][0])
        loc = metadata['leak']['features'][0]['geometry']['coordinates']
        minx=bbox[:,0].min()
        maxx=bbox[:,0].max()
        miny=bbox[:,1].min()
        maxy=bbox[:,1].max()
        bbox_shp = shapely.geometry.box(
            minx=minx,
            maxx=maxx,
            miny=miny,
            maxy=maxy
        )
        loc_shp = shapely.geometry.Point(loc[0],loc[1])
        shapely.geometry.GeometryCollection([bbox_shp, loc_shp])
        try:
            assert loc_shp.intersects(bbox_shp), 'leak location should be inside image'
            assert bbox_shp.centroid.almost_equals(loc_shp, decimal=5), 'leak should be near center of image'
            assert (np.array([d.shape for d in data])==pixel_length).all(), 'image area should be the right amount of pixels'
            assert (maxx-minx)/(maxy-miny)<1.3, 'should be roughly square'
            assert (maxx-minx)/(maxy-miny)>0.7, 'should be roughly square'
            assert not empty_bands.all(), 'should not have all bands empty'
        except Exception as exc:
            print(path, exc)
#             raise(exc)
            discarded.append(path)
        else:
            X.append(data)
            y.append(yy)
            t.append(tt)
            m.append(metadata)
        

len(X), len(discarded)

(15211, 0)

In [140]:
# shuffle
from sklearn.utils import shuffle
X,y,m= shuffle(X,y,m,random_state=1337)

In [141]:
# which empty bands do we have?
a=np.array([x.sum(-1).sum(-1)==0 for x in X])
print('amount of each band',list(zip(bands_l8,a.sum(0))))
print('mean amount of bands',a.sum(1).mean())

amount of each band [('B1', 0), ('B2', 0), ('B3', 0), ('B4', 0), ('B5', 0), ('B6', 0), ('B7', 0), ('B8', 0), ('B9', 0), ('B10', 0), ('B11', 0), ('BQA', 0)]
mean amount of bands 0.0


In [142]:
# save using hdf5 (so keras can easily load it) and json 
import h5py
h5file = output_dir.joinpath('data.h5')
with h5py.File(h5file, 'w') as h5f:
    h5f.create_dataset('X', data=X)
    h5f.create_dataset('y', data=y)

json.dump(m,open(output_dir.joinpath('data_metadata.json'),'w'))

with open(output_dir.joinpath('readme.md'),'w') as fo:
    fo. write("""
Files:
- cache- cached tiff files
- script_metadata.json - information on scraping script
- data.h5 contains X, y, and t.
    - X: tiff files for each band loaded into an array of shape (Leak, Bands, width, length)
    - y: True for before the leak, False for after
- data_metadata: array of metadata for each leak in X. Each contain info on leak, image, and image search
    
Loading: 
```py
# load
metadatas = json.load(open('data_metadata.json'))
with h5py.File('data.h5','r') as h5f:
    X2 = h5f['X'][:]
    y2 = h5f['y'][:]
y
```
    """)

In [139]:
# test load
metadatas = json.load(open(output_dir.joinpath('data_metadata.json')))
with h5py.File(output_dir.joinpath('data.h5'),'r') as h5f:
    X2 = h5f['X'][:]
    y2 = h5f['y'][:]
X2.shape, y2, metadatas[0].keys()

((15211, 12, 25, 25),
 array([False,  True, False, ...,  True,  True, False], dtype=bool),
 dict_keys(['name', 'leak', 'image', 'distance', 'scale', 'crs']))

# test deleteme

In [82]:
os.environ['CUDA_VISIBLE_DEVICES']="" # to disable gpu, so I can do large predictions in memory

helper_dir = str(Path('.').abspath())
if helper_dir not in os.sys.path:
    os.sys.path.append(helper_dir)

from leak_helpers.earth_engine import display_ee, get_boundary, tifs2np, bands_s2, download_image, bands_s2
from leak_helpers.geometry import diffxy, resample_polygon
from leak_helpers.modelling import ImageDataGenerator, dice_coef_loss
from leak_helpers.visualization import imshow_bands
from leak_helpers.analysis import parse_classification_report, find_best_dummy_classification, calculate_result_class
from leak_helpers.modelling.filters import is_not_cloudy, is_not_center_cloudy, is_image_within, is_leak, filter_split_data, is_not_dup, hash_rows, normalise_bands

Using TensorFlow backend.


In [83]:
md=m[2]
t_image = arrow.get(md['image']['properties']['system:time_end'] / 1000)
t_leak = arrow.get(md['leak']['features'][0]['properties']['REPO_Date'])
seconds_before_leak = (t_leak - t_image).total_seconds()
seconds_before_leak#<60*60*24*4
# [is_image_within(mm, 60*60*24*4) for mm in m]

-371640.0380

In [84]:
# has_three_bands = np.array([x.sum(-1).sum(-1)==0 for x in X]).sum(1)>2


# X = X[keep]
# y = y[keep]
# m = [m[i] for i in range(len(m)) if keep[i]]
# print('kept',keep.sum(),'of',len(keep))


In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, metadata_train, metadata_test = train_test_split(
        X, y, m)

X_train2 = X_train.reshape((len(X_train),-1))
X_test2 = X_test.reshape((len(X_test),-1))

In [86]:
target_names = ['noleak','leak']
df_dummies, best_dummy = find_best_dummy_classification(X,y,n=50, target_names=target_names)

/home/isisilon/.virtualenvs/py3syspck/lib/python3.4/site-packages/sklearn/metrics/classification.py:516: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(var_yt * var_yp)
/home/isisilon/.virtualenvs/py3syspck/lib/python3.4/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [87]:
import sklearn.ensemble
thresh=0.5
clf = sklearn.ensemble.RandomForestClassifier(
    n_estimators=200, 
#     criterion='entropy',
#     max_depth=None, 
    min_samples_split=6, 
    min_samples_leaf=6,
#     max_features='auto', 
#     bootstrap=True,
#         random_state=0,
#     n_jobs=4, 
)

clf.fit(X_train2, y_train) 

y_pred = clf.predict(X_test2)
score = clf.score(X_test2, y_test)

matthews_corrcoef = sklearn.metrics.matthews_corrcoef(y_test>thresh, y_pred>thresh)
print(sklearn.metrics.classification_report(y_test > thresh, y_pred > thresh, target_names=target_names))
score,matthews_corrcoef

             precision    recall  f1-score   support

     noleak       0.57      0.55      0.56       618
       leak       0.59      0.61      0.60       663

avg / total       0.58      0.58      0.58      1281



(0.5792, 0.1565)